In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, kstest
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

## WHO Dataset :
### Describtion:


### *1) id: unique identifier*

### *2) gender: "Male", "Female" or "Other"*

### *3) age: age of the patient**

### *4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension*

### *5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease*

### *6) ever_married: "No" or "Yes"*

### *7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"*

### *8) Residence_type: "Rural" or "Urban"*

### *9) avg_glucose_level: average glucose level in blood*

### *10) bmi: body mass index*

### *11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"**

### *12) stroke: 1 if the patient had a stroke or 0 if not*

*Note: "Unknown" in smoking_status means that the information is unavailable for this patient*


In [ ]:
data = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
data

## Let's Check Null Value :
### Only Bmi Has Null Value

In [ ]:
data.info()

In [ ]:
data['bmi'] = data['bmi'].fillna(data['bmi'].mean())

In [ ]:
data.isnull().sum().plot(kind='bar')

In [ ]:
data['id'].duplicated().sum()

In [ ]:
sns.distplot(data['age'], bins=100);

In [ ]:
# Bernoli Distribution 
sns.distplot(data['stroke']);

In [ ]:
# is it normal or not ?
## let's check it 
sns.distplot(data['avg_glucose_level'], bins=100);

In [ ]:
def kl(p, q):
    res = np.sum(np.where(np.logical_and( p!=0 ,q!=0), p * np.log(p / q), 0))
    return res

In [ ]:
count, devision = np.histogram(data['avg_glucose_level'], bins=100)

In [ ]:
ideal = norm.rvs(size=len(data['avg_glucose_level']), loc = data['avg_glucose_level'].mean(), scale =  data['avg_glucose_level'].std())
icount, idevision = np.histogram(ideal, bins=100)

In [ ]:
kl(count/5110, icount/5110)

In [ ]:
plt.plot(np.cumsum(data['avg_glucose_level'].sort_values().reset_index(drop=True)))
plt.plot(np.cumsum(ideal), color='red')

In [ ]:
data['bmi'].plot(kind='hist', bins=100)

In [ ]:
ideal = norm.rvs(size=len(data['bmi']), loc = data['bmi'].mean(), scale =  data['bmi'].std())

In [ ]:
plt.hist(ideal, bins=100)
plt.hist(data['bmi'], bins=100)

In [ ]:
sns.countplot(data['gender']);

## Clean The Data:

### ** When i want to try remove the Other on gender column i got error ** 
### so try to another way to remove it 

In [ ]:
data.loc[data['gender'].str.contains('Other')]

In [ ]:
data[data.gender.values == 'Other']

In [ ]:
def remove(gender):
    if 'Other' in gender:
        return np.nan
    else:
        return gender

In [ ]:
data['gender'] = data['gender'].apply(remove)
data['gender'] = data['gender'].dropna()

In [ ]:
sns.countplot(data['gender']);

In [ ]:
data = data.drop('id', axis=1)

In [ ]:
sns.countplot(data['smoking_status']);

In [ ]:
data['smoking_status'].value_counts()

In [ ]:
from numpy import nan as NA
#data[data['age'] < 20]
def remove_age(age):
    if age < 18:
        return NA
    else:
        return age

In [ ]:
data['age'] = data['age'].apply(remove_age)
data['age'].dropna(inplace=True)
data.dropna(inplace=True)

In [ ]:
data

In [ ]:
data['age'].describe()

### Children (00-14 years)
### Youth (15-24 years)
### Adults (25-64 years)
### Seniors (65 years and over)

In [ ]:
data['categori_age'] = pd.cut((data['age']), 3, labels=['Youth ', 'Adults ', 'Seniors '])

In [ ]:
obj = data.select_dtypes(include='object')
obj

In [ ]:
data

##  Now We Want To See If The Age Is More Than 40 or 50 Is Really Effected On Stroke Or Not
### if p_value < 0.5 means we can have a reason to prove it if the p_value > 0.5 means maybe it happens base on chance

In [ ]:
from numpy import sqrt, round, abs

In [ ]:
dataA = data[data['age'] >= 50]['stroke']
dataB = data[data['age'] < 50]['stroke']

In [ ]:
Amean = dataA.mean()
Bmean = dataB.mean()
Astd = dataA.std()
Bstd = dataB.std()
Alen = len(dataA)
Blen = len(dataB)

In [ ]:
above = Amean - Bmean
below = sqrt((Astd ** 2 / Alen) + (Bstd ** 2 / Blen))
z  = above / below 

In [ ]:
z

In [ ]:
p_value = 2 * (1 - norm.cdf(abs(z)))
p_value

## Anova_Test

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [ ]:
res = ols('stroke ~ age', data=data).fit()
sm.stats.anova_lm(res)

In [ ]:
data

In [ ]:
data.groupby(['gender', 'categori_age'])['stroke'].size()

In [ ]:
data.pivot_table(values='age', columns='categori_age', index='stroke', aggfunc='count').plot(kind='bar', figsize=(15, 8))

In [ ]:
data.groupby(['Residence_type', 'work_type'])['stroke'].size().plot(kind='bar')

In [ ]:
data

In [ ]:
sns.catplot(
    data=data, kind="bar",
    x="gender", y="heart_disease", hue="stroke",
    ci="sd", palette="dark"
)

In [ ]:
plt.figure(figsize=(10, 10))
plt.hlines(40, 20, 80, color='black')
plt.scatter(x=data['age'], y=data['bmi'], color='red')

# Convert To Numerical:

In [ ]:
data['gender'] = pd.get_dummies(data['gender'])
data['Residence_type'] = pd.get_dummies(data['Residence_type'])
data['ever_married'] = pd.get_dummies(data['ever_married'])
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer
##                              *** For Machine Learning use ohe ***
#ohe = make_column_transformer((OneHotEncoder(), ['work_type', 'smoking_status', 'categori_age']), remainder='passthrough')
#data = ohe.fit_transform(data)

In [ ]:
label = LabelEncoder()
data['work_type'] = label.fit_transform(data['work_type'])
data['smoking_status'] = label.fit_transform(data['smoking_status'])
data['categori_age'] = label.fit_transform(data['categori_age'])


In [ ]:
data.drop('age', inplace=True, axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
stn = StandardScaler()
data['avg_glucose_level'] = stn.fit_transform(data['avg_glucose_level'].values.reshape(-1, 1))
data['bmi'] = stn.fit_transform(data['bmi'].values.reshape(-1, 1))


In [ ]:
data

# Feature Selcetion
## Chi2

In [ ]:
from sklearn.feature_selection import chi2, SelectKBest

x = data.drop('stroke', axis=1)
y =  data['stroke']
best_f = SelectKBest(chi2, k=5)
best_f.fit_transform(abs(x), y)


dfscores = pd.DataFrame(best_f.scores_)
dfcolumns = pd.DataFrame(x.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['feature','score']  
print(featureScores.nlargest(5,'score')) 

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(x.corr(), annot=True)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
from  sklearn.metrics import confusion_matrix

# Random_Forest:


In [ ]:
from sklearn.ensemble  import RandomForestClassifier
clf_r = RandomForestClassifier(n_estimators=10)
clf_r.fit(x_train, y_train)
print(clf_r.score(x_train, y_train))
r_predict = clf_r.predict(x_test)
# Evaluate Model:
print(confusion_matrix(y_test, r_predict))

# Logestic_regression:

In [ ]:
from  sklearn.linear_model import LogisticRegression
clf_l = LogisticRegression()
clf_l.fit(x_train, y_train)
print(clf_l.score(x_train, y_train))
l_predict = clf_l.predict(x_test)
# Evaluate Model:
print(confusion_matrix(y_test, l_predict))

# SVM:

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'poly'), 'C':[1,2, 3, 4, 5]}

svc = SVC()
clf_sg = GridSearchCV(svc, parameters)
clf_sg.fit(x_train, y_train)

print(clf_sg.best_estimator_)
print(clf_sg.score(x_train, y_train))

sg_predict = clf_sg.predict(x_test)
# Evaluate Model:
print(confusion_matrix(y_test, sg_predict))

# PCA:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
x2d = pca.fit_transform(x_train)

In [ ]:
plt.scatter(x = x2d[:, 0], y = x2d[:, 1])

In [ ]:
pca.explained_variance_ratio_

# Another Way To Feature Selection:
### use Boruta 
#### See How It's Work



Faster run times, thanks to scikit-learn

Scikit-learn like interface

Compatible with any ensemble method from scikit-learn

Automatic n_estimator selection

Ranking of features

Feature importances are derived from Gini impurity instead of RandomForest R package's MDA

In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier()
from boruta import BorutaPy
feature = BorutaPy(model, n_estimators='auto', verbose=2, random_state=1)

## Pay attention Use Np.Array Is Not Mandetory

In [ ]:
feature.fit(np.array(x_train), np.array(y_train))

In [ ]:
print(feature.support_)
print(feature.ranking_)
print(x_train.columns[feature.support_].to_list())

In [ ]:
x_boruta = feature.transform(np.array(x_train))

In [ ]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_boruta, y_train)

In [ ]:
x_test_filtered = feature.transform(np.array(x_test))
prediction_xgb = xgb_model.predict(x_test_filtered)

In [ ]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction_xgb))

In [ ]:
cm = confusion_matrix(y_test, prediction_xgb)
#print(cm)
sns.heatmap(cm, annot=True)